In [1]:
:set -fno-ghci-sandbox

In [2]:
{-# LANGUAGE TemplateHaskell #-}
{-# LANGUAGE QuasiQuotes #-}

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE TemplateHaskell #-}
Why not:
Line 2: Unused LANGUAGE pragma
Found:
{-# LANGUAGE QuasiQuotes #-}
Why not:

In [3]:
:show modules

: 

In [4]:
import qualified Language.R as R
import Language.R.QQ

In [5]:
:t R.runRegion

R.runRegion :: forall a. NFData a => (forall s. R s a) -> IO a

In [6]:
:t [r| print(pi) |]

[r| print(pi) |] :: forall (m :: * -> *). MonadR m => m (SomeSEXP (PrimState m))

In [7]:
(R.runRegion $ do _ <- [r| print(pi) |]; return ())

[1] 3.141593

In [8]:
R.runRegion $ do x <- [r| loadedNamespaces() |]; [r| print(x_hs) |]; return ()

[1] "compiler"  "graphics"  "utils"     "grDevices" "stats"     "datasets" 
[7] "methods"   "base"

In [9]:
R.runRegion $ do x <- [r| library(cmaes) |]; [r| print(x_hs) |]; return ()

[1] "cmaes"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"

In [10]:
R.runRegion $ do x <- [r| loadedNamespaces() |]; [r| print(x_hs) |]; return ()

[1] "compiler"  "graphics"  "utils"     "grDevices" "stats"     "datasets" 
[7] "cmaes"     "methods"   "base"

In [11]:
R.runRegion $ do x <- [r| cma_es(c(0, 0), function(x, ...) drop(crossprod(x)), lower=rep(-10, 2), upper=rep(10, 2), control=list(stopfitness=1e-5, maxit=400)) |]; [r| print(x_hs) |]; return ()

$par
[1]  0.001400161 -0.002712781

$value
[1] 9.319633e-06

$counts
function gradient 
     102       NA 

$convergence
[1] 0

$message
[1] "Stop fitness reached."

$constr.violations
[1] 0

$diagnostic
list()

attr(,"class")
[1] "cma_es.result"

In [12]:
:set -XLambdaCase
:set -XDataKinds
:set -XGADTs
:set -XFlexibleContexts
:set -XScopedTypeVariables

In [13]:
import qualified Language.R.Matcher as M
import qualified Data.Vector.SEXP as V
import qualified Language.R.HExp as H
import           Language.R (R)

In [14]:
int2limVV :: forall s . ([Double] -> Double) -> [Double] -> [Double] -> R s Double
int2limVV g l h = do
  ss <- [r| hcubature (function (x) matrix(f_hs(x)),
                       lowerLimit = l_hs,
                       upperLimit = h_hs,
                       maxEval = 0,
                       vectorInterface=TRUE) |]
  R.fromSEXP . R.cast R.SReal <$> [r| ss_hs$integral |]
  where
    f :: R.SEXP s 'R.Real -> R s [Double]
    f w = M.matchOnly go (R.SomeSEXP w) >>= \case
            Left e -> error (show e)
            Right xss -> pure $ map g xss
      where
        go = do
           M.hexp R.SReal $ \(H.Real v) -> do
              foo <- M.dim
              case foo of
                [n,m] -> pure $ map (\jj -> map (\ii -> v V.! (jj * n + ii)) [0..n-1]) [0..m-1]
                otherwise -> error "foo"

In [15]:
r0 = R.runRegion $ do
  _ <- [r| require(cubature) |]
  let g :: [Double] -> Double
      g [a, b] = exp(negate (a*a + b*b) / 2) / (2 * pi)
      g xs     = error $ show xs
  int2limVV g [(-3.0), (-3.0)] [3.0, 3.0]

In [16]:
r0

Loading required package: cubature
0.9946063639261128

In [17]:
cmaes :: forall s . ([Double] -> Double) -> [Double] -> [Double] -> [Double] -> R s (Double, [Double])
cmaes g i l h = do
  ss <- [r| cma_es (function (x) f_hs(x),
                       par = i_hs,
                       lower = l_hs,
                       upper = h_hs,
                       control=list(stopfitness=1e-5, maxit=400)) |]
  pars :: [Double] <- R.dynSEXP <$> [r| ss_hs$par |]
  value :: Double <- R.dynSEXP <$> [r| ss_hs$value |]
  return (value, pars)
  where
    f :: R.SEXP s 'R.Real -> R s Double
    f w = M.matchOnly go (R.SomeSEXP w) >>= \case
            Left e -> error ("You are here: " ++ show e)
            Right xs -> pure $ g xs
      where
        go :: M.Matcher s [Double]
        go = do
           M.hexp R.SReal $ \(H.Real v) -> do
              return $ V.toList v

In [18]:
  r0 <- R.runRegion $ do
    baz <- [r| require(cmaes) |]
    let g :: [Double] -> Double
        g [a, b] = a*a + b*b
        g xs     = error $ show xs
    cmaes g [0.0, 0.0] [(-10.0), (-10.0)] [10.0, 10.0]
  print r0

(6.313306365090975e-6,[2.467427198716352e-3,4.744569360076329e-4])

In [19]:
  r1 <- R.runRegion $ do
    _ <- [r| require(cmaes) |]
    let g :: [Double] -> Double
        g [a, b, c] = a*a + b*b + c*c
        g xs     = error $ show xs
    cmaes g [0.0, 0.0, 0.0] [(-10.0), (-10.0), (-10.0)] [10.0, 10.0, 10.0]
  print r1

(4.897433410053955e-6,[1.3183012007430168e-3,-1.5369706646365406e-3,8.928810280323949e-4])

# Example

We wish to see if when we are given enough samples from a multivariate normal distribution together with their probability density then we can infer the parameters of the generating distribution using CMAES.

We need to have a well formed covariance matrix

$$\mathrm{K}_{\mathrm{XX}}=\operatorname{cov}[\mathbf{X}, \mathbf{X}]=\mathrm{E}\left[\left(\mathbf{X}-\mu_{\mathbf{X}}\right)\left(\mathbf{X}-\mu_{\mathbf{X}}\right)^{\mathrm{T}}\right]=\mathbf{E}\left[\mathbf{X} \mathbf{X}^{T}\right]-\mu_{\mathbf{X}} \mu_{\mathbf{X}}^{T}$$

We can generate one of these from a vector of variances and a
 correlation matrix. Recall that in a correlation matrix all the
 diaganols are $1$ and the other entries are in $[-1,1]$.

$$\operatorname{corr}(\mathbf{X})=\left(\operatorname{diag}\left(\mathbf{K}_{\mathbf{X} \mathbf{X}}\right)\right)^{-\frac{1}{2}} \mathbf{K}_{\mathbf{X X}}\left(\operatorname{diag}\left(\mathbf{K}_{\mathbf{X} \mathbf{X}}\right)\right)^{-\frac{1}{2}}$$

$$\sigma^2 = \operatorname{diag}(\mathbf{K}_{\mathbf{XX}})$$

$$\rho = \operatorname{corr}(\mathbf{X})$$

$$\rho = \left(\operatorname{diag}\left(\sigma^2\right)^{-\frac{1}{2}}\right)
         \mathbf{K}_{\mathbf{X X}}
         \left(\operatorname{diag}\left(\sigma^2)\right)^{-\frac{1}{2}}\right)$$

$$\mathbf{K}_{\mathbf{XX}} = \operatorname{diag}(\mathbf{\sigma})\operatorname{corr}(\mathbf{X})\operatorname{diag}(\mathbf{\sigma})$$

In [20]:
import GHC.TypeLits
import Data.Proxy

In [21]:
import qualified Numeric.LinearAlgebra.Static as LS
import Prelude hiding ((<>))
import Numeric.LinearAlgebra.Static ((<>))

In [22]:
corrToCov :: KnownNat n => LS.R n -> LS.Sq n -> LS.Sq n
corrToCov sigma rho = kHalf <> rho <> kHalf
  where
    kHalf = LS.diag sigma

Now we can create example samples from a bivariate normal. Note that
 we fix the means to be $0$ but this could be generalized
 straightforwardly.

In [23]:
:set -XDataKinds

In [24]:
import qualified Data.Random as R
import Data.Random.Source.PureMT
import Control.Monad.State
import Data.Random.Distribution.Static.MultivariateNormal

In [25]:
nSamples :: Int
nSamples = 1000

sigma1, sigma2, rho12 :: Double
sigma1 = 3.0
sigma2 = 1.0
rho12 = 0.5

singleSample :: R.RVarT (State PureMT) (LS.R 2)
singleSample = R.sample $ Normal (LS.vector [0.0, 0.0])
               (LS.sym $ LS.matrix [ sigma1, rho12 * sqrt sigma1 * sqrt sigma2
                                   , rho12 * sqrt sigma1 * sqrt sigma2, sigma2])

multiSamples :: [LS.R 2]
multiSamples = evalState (replicateM nSamples $ R.sample singleSample) (pureMT 3)

We can generalize this to arbitrary dimensions.

In [26]:
import qualified Numeric.LinearAlgebra as LA
import Data.Maybe ( fromJust )

In [27]:
singleSampleN :: forall n . KnownNat n =>
                 LS.R n -> LS.Sq n -> R.RVarT (State PureMT) (LS.R n)
singleSampleN vars corrs = R.sample $ Normal (LS.vector ds) es
  where
    sigmas :: LS.R n
    sigmas = fromJust $ LS.create $ LA.cmap sqrt $ LS.extract vars

    es :: LS.Sym n
    es = LS.sym $ corrToCov sigmas corrs
    ds = replicate (fromIntegral $ natVal (Proxy :: Proxy n)) 0.0

multiSamplesN :: forall n . KnownNat n =>
                  LS.R n -> LS.Sq n -> [LS.R n]
multiSamplesN x y = evalState (replicateM nSamples $ R.sample (singleSampleN x y)) (pureMT 3)

We can generate some samples from the bivariate normal using our specialised function.

In [28]:
pts :: [(Double, Double)]
pts = map f multiSamples
  where
    f z = (x, y)
      where
        (x, t) = LS.headTail z
        (y, _) = LS.headTail t

Let's check our generalized function matches.

In [29]:
pts2' :: [(Double, Double)]
pts2' = map f ((multiSamplesN (LS.vector [sigma1, sigma2]) (LS.matrix [1.0, rho12, rho12, 1.0])) :: [LS.R 2])
  where
    f z = (x, y)
      where
        (x, t) = LS.headTail z
        (y, _) = LS.headTail t

Line 2: Redundant bracket
Found:
  (multiSamplesN
     (LS.vector [sigma1, sigma2])
     (LS.matrix [1.0, rho12, rho12, 1.0])) ::
    [LS.R 2]
Why not:
  multiSamplesN
    (LS.vector [sigma1, sigma2])
    (LS.matrix [1.0, rho12, rho12, 1.0]) ::
    [LS.R 2]

In [30]:
last pts

(-4.065769711374732,8.721890276573241e-2)

In [31]:
last pts2'

(-4.065769711374731,8.721890276573241e-2)

We also need to calculate the probability density for each sample.

In [32]:
import Data.Random.Distribution

In [33]:
pdfs :: [Double]
pdfs = map (pdf (Normal (LS.fromList [0.0, 0.0])
                        (LS.sym $ LS.matrix [ sigma1, rho12 * sqrt sigma1 * sqrt sigma2
                                            , rho12 * sqrt sigma1 * sqrt sigma2, sigma2]))) multiSamples

So the density for $\texttt{last pts2'}$ is $\texttt{last pdfs}$.

In [34]:
last pdfs

2.3382685607071917e-3

Let's check a few more things. First let's create the covariance
 matrix from the correlation matrix rather than doing it by hand as
 above.

In [35]:
covv :: LS.Sq 2
covv = corrToCov (LS.vector [sqrt sigma1, sqrt sigma2])
                 (LS.matrix [1.0, rho12, rho12, 1.0])

The covariance matrix is

In [36]:
covv

(matrix
 [ 2.9999999999999996, 0.8660254037844386
 , 0.8660254037844386,                1.0 ] :: L 2 2)

Let's check it gives the same answer.

In [37]:
pdfs' :: [Double]
pdfs' = map (pdf (Normal (LS.fromList [0.0, 0.0]) (LS.sym covv))) multiSamples

In [38]:
last pdfs

2.3382685607071917e-3

In [39]:
last pdfs'

2.3382685607071917e-3

Again we can generalize this.

In [40]:
pdfsN :: forall n . KnownNat n =>
          LS.R n -> LS.Sq n ->  LS.Sym n -> [Double]
pdfsN x y cov = map (pdf (Normal (LS.fromList ds) cov)) (multiSamplesN x y)
  where
    ds = replicate (fromIntegral $ natVal (Proxy :: Proxy n)) 0.0

And test that this also gives the same answer when specialized.

In [41]:
test :: [Double]
test = zipWith (-) a b
  where
    a = pdfsN (LS.fromList [sigma1, sigma2]) (LS.matrix [1.0, rho12, rho12, 1.0]) (LS.sym covv)
    b = pdfs'

In [42]:
maximum $ map abs test

4.163336342344337e-17

In [43]:
costNorm :: [(Double, Double)] -> (Double, Double) -> (Double, Double) -> Double -> Double
costNorm obs (muX, muY) (varX, varY) tau =
  sum $ zipWith (\x y -> (x - y) * (x - y)) pdfsOfObs pdfs
  where
    t2 :: LS.Sym 2
    t2 = LS.sym $ corrToCov (LS.fromList [sqrt varX, sqrt varY]) (LS.matrix [1.0, tau, tau, 1.0])

    cost1 :: LS.R 2 -> Double
    cost1 = pdf (Normal (LS.fromList [muX, muY]) t2)

    pdfsOfObs :: [Double]
    pdfsOfObs = map cost1 $ map (\(x, y) -> LS.fromList [x, y]) obs

Line 12: Use map once
Found:
map cost1 $ map (\ (x, y) -> LS.fromList [x, y]) obs
Why not:
map (cost1 . (\ (x, y) -> LS.fromList [x, y])) obs

In [44]:
costNorm2 :: [LS.R 2] -> LS.R 2 -> LS.R 2 -> LS.Sym 2 -> Double
costNorm2 obs mus vars corr =
  sum $ zipWith (\x y -> (x - y) * (x - y)) pdfsOfObs pdfs
  where
    sigmas :: LS.R 2
    sigmas = fromJust $ LS.create $ LA.cmap sqrt $ LS.extract vars

    t2 :: LS.Sym 2
    t2 = LS.sym $ corrToCov sigmas (LS.unSym corr)

    getPdf :: LS.R 2 -> Double
    getPdf = pdf (Normal mus t2)

    pdfsOfObs :: [Double]
    pdfsOfObs = map getPdf obs

And then generalize fully.

In [45]:
costNormN :: forall n . KnownNat n =>
             [LS.R n] -> LS.R n ->
             LS.R n -> LS.Sym n ->
             LS.R n -> LS.Sym n -> Double
costNormN obs mus possVars possCorrs genVars genCorrs =
  sum $ zipWith (\x y -> (x - y) * (x - y)) pdfsOfObs pdfsOfGen
  where
    sigmas :: LS.R n
    sigmas = fromJust $ LS.create $ LA.cmap sqrt $ LS.extract possVars

    t2 :: LS.Sym n
    t2 = LS.sym $ corrToCov sigmas (LS.unSym possCorrs)

    getPdf :: LS.R n -> Double
    getPdf = pdf (Normal mus t2)

    pdfsOfObs :: [Double]
    pdfsOfObs = map getPdf obs

    pdfsOfGen = pdfsN genVars (LS.unSym genCorrs) cov

    taus :: LS.R n
    taus = fromJust $ LS.create $ LA.cmap sqrt $ LS.extract genVars

    cov :: LS.Sym n
    cov = LS.sym $ corrToCov taus (LS.unSym genCorrs)

In [46]:
infer2 :: [(Double, Double)] -> R s (Double, [Double])
infer2 obs = cmaes co[0.0, 0.0, log 3.0, log 1.5, 0.5] [(-1.0), (-1.0), log 1.0, log 1.0, 0.1] [1.0, 1.0, log 5.0, log 3.0, 0.9]
  where
    co [x1, x2, x3, x4, x5] =
      costNorm2 (map (\(x, y) -> LS.fromList [x, y]) obs)
                 (LS.fromList [x1, x2])
                 varV
                 corrM
      where
        varV :: LS.R 2
        varV   = LS.fromList [exp x3, exp x4]
        corrM :: LS.Sym 2
        corrM  = LS.sym $
                 LS.matrix [1.0, ((2 / pi) * atan x5),
                            ((2 / pi) * atan x5), 1.0]
    co _                    =
      error "infer2 wrong number of elements"

Line 2: Redundant bracket
Found:
[(- 1.0), (- 1.0), log 1.0, log 1.0, 0.1]
Why not:
[- 1.0, (- 1.0), log 1.0, log 1.0, 0.1]Line 2: Redundant bracket
Found:
[(- 1.0), (- 1.0), log 1.0, log 1.0, 0.1]
Why not:
[(- 1.0), - 1.0, log 1.0, log 1.0, 0.1]Line 14: Redundant bracket
Found:
[1.0, ((2 / pi) * atan x5), ((2 / pi) * atan x5), 1.0]
Why not:
[1.0, (2 / pi) * atan x5, ((2 / pi) * atan x5), 1.0]Line 14: Redundant bracket
Found:
[1.0, ((2 / pi) * atan x5), ((2 / pi) * atan x5), 1.0]
Why not:
[1.0, ((2 / pi) * atan x5), (2 / pi) * atan x5, 1.0]

In [47]:
result <- R.runRegion (infer2 pts)

In [50]:
exp (snd result!!2)

2.8988529833602406

In [52]:
exp (snd result!!3)

0.9999999961497585

In [55]:
atan (snd result!!4) * 2 / pi

0.4665245863550214